In [ ]:
import random
import copy
import sys

# Configuration du problème left move
DEPTH = 10  # Profondeur de l'arbre de recherche
MAX_LEVEL = 3  # Niveau maximum de NMCS à tester

# ===== FONCTIONS DE BASE POUR LE PROBLÈME LEFT MOVE =====

def legalMoves(state):
    """
    Retourne les mouvements légaux depuis un état.
    Dans le problème left move : 0 = gauche, 1 = droite
    """
    if terminal(state):
        return []
    return [0, 1]

def play(state, move):
    """
    Applique un mouvement à un état et retourne le nouvel état.
    """
    new_state = copy.deepcopy(state)
    new_state.append(move)
    return new_state

def terminal(state):
    """
    Vérifie si un état est terminal (profondeur maximale atteinte).
    """
    return len(state) >= DEPTH

def score(state):
    """
    Calcule le score d'un état.
    Dans le problème left move : compte le nombre de mouvements à gauche (0).
    """
    return sum(1 for move in state if move == 0)

def playout(state):
    """
    Effectue un playout aléatoire depuis un état jusqu'à un état terminal.
    """
    current_state = copy.deepcopy(state)
    
    while not terminal(current_state):
        moves = legalMoves(current_state)
        if not moves:
            break
        move = random.choice(moves)
        current_state = play(current_state, move)
    
    return current_state

# ===== ALGORITHME NESTED MONTE CARLO SEARCH =====

# Variable globale pour mémoriser la meilleure séquence
best_sequence = []
best_score = -1

def nested(state, level):
    """
    Algorithme Nested Monte Carlo Search.
    
    Args:
        state: État actuel du jeu
        level: Niveau de profondeur de la recherche imbriquée
        
    Returns:
        État final après la recherche
    """
    global best_sequence, best_score
    
    current_state = copy.deepcopy(state)
    
    # Si level = 0, effectuer un playout aléatoire
    if level == 0:
        return playout(current_state)
    
    # Mémorisation locale de la meilleure séquence pour ce niveau
    local_best_sequence = []
    local_best_score = -1
    
    while not terminal(current_state):
        moves = legalMoves(current_state)
        if not moves:
            break
            
        best_move = moves[0]
        best_move_score = -1
        best_move_sequence = []
        
        # Essayer chaque mouvement possible
        for move in moves:
            # Jouer le mouvement
            new_state = play(current_state, move)
            
            # Recherche imbriquée au niveau inférieur
            final_state = nested(new_state, level - 1)
            move_score = score(final_state)
            
            # Garder le meilleur mouvement
            if move_score > best_move_score:
                best_move_score = move_score
                best_move = move
                best_move_sequence = final_state
        
        # Mise à jour de la meilleure séquence locale
        if best_move_score > local_best_score:
            local_best_score = best_move_score
            local_best_sequence = best_move_sequence
        
        # Utiliser la meilleure séquence mémorisée si elle existe et est meilleure
        if len(best_sequence) > len(current_state) and score(best_sequence) >= best_move_score:
            next_move = best_sequence[len(current_state)]
            current_state = play(current_state, next_move)
        else:
            # Sinon, jouer le meilleur mouvement trouvé
            current_state = play(current_state, best_move)
            # Mettre à jour la meilleure séquence globale
            if local_best_score > best_score:
                best_sequence = local_best_sequence
                best_score = local_best_score
    
    return current_state

def iterative_nested(initial_state, level, max_iterations=1000):
    """
    Version itérative de Nested Monte Carlo Search.
    """
    global best_sequence, best_score
    
    best_sequence = []
    best_score = -1
    best_iteration_score = -1
    best_result = None
    
    print(f"Démarrage de NMCS niveau {level} avec {max_iterations} itérations...")
    
    for iteration in range(max_iterations):
        # Réinitialiser pour chaque itération
        result_state = nested(copy.deepcopy(initial_state), level)
        iteration_score = score(result_state)
        
        if iteration_score > best_iteration_score:
            best_iteration_score = iteration_score
            best_result = result_state
            print(f"Itération {iteration+1}: Nouveau meilleur score = {iteration_score}")
            print(f"Séquence: {result_state}")
        
        # Affichage périodique
        if (iteration + 1) % 100 == 0:
            print(f"Itération {iteration+1}: Meilleur score actuel = {best_iteration_score}")
    
    return best_result, best_iteration_score

# ===== FONCTION DE TEST ET COMPARAISON =====

def test_all_methods():
    """
    Teste et compare différentes méthodes sur le problème left move.
    """
    initial_state = []
    num_tests = 100
    
    print("="*60)
    print(f"TEST DU PROBLÈME LEFT MOVE (profondeur {DEPTH})")
    print("="*60)
    
    # Test du playout aléatoire
    print("\n1. PLAYOUT ALÉATOIRE:")
    random_scores = []
    for _ in range(num_tests):
        result = playout(initial_state)
        random_scores.append(score(result))
    
    avg_random = sum(random_scores) / len(random_scores)
    max_random = max(random_scores)
    print(f"Score moyen: {avg_random:.2f}")
    print(f"Meilleur score: {max_random}")
    
    # Test de NMCS pour différents niveaux
    for level in range(1, MAX_LEVEL + 1):
        print(f"\n{level + 1}. NESTED MONTE CARLO SEARCH (Niveau {level}):")
        
        # Réinitialiser les variables globales
        global best_sequence, best_score
        best_sequence = []
        best_score = -1
        
        # Tester plusieurs fois pour avoir une moyenne
        nmcs_scores = []
        for test in range(min(10, num_tests // 10)):  # Moins de tests car plus coûteux
            result, result_score = iterative_nested(initial_state, level, max_iterations=50)
            nmcs_scores.append(result_score)
            if test == 0:  # Afficher le détail du premier test
                print(f"Première séquence trouvée: {result}")
        
        avg_nmcs = sum(nmcs_scores) / len(nmcs_scores)
        max_nmcs = max(nmcs_scores)
        print(f"Score moyen: {avg_nmcs:.2f}")
        print(f"Meilleur score: {max_nmcs}")
        print(f"Amélioration vs aléatoire: {((avg_nmcs - avg_random) / avg_random * 100):.1f}%")

# ===== DÉMONSTRATION INTERACTIVE =====

def demo_single_search():
    """
    Démonstration d'une recherche unique avec visualisation.
    """
    print("="*60)
    print("DÉMONSTRATION INTERACTIVE")
    print("="*60)
    
    initial_state = []
    level = 2
    
    print(f"Recherche NMCS niveau {level} pour le problème left move...")
    print(f"Objectif: maximiser le nombre de mouvements à gauche (0)")
    print(f"Profondeur: {DEPTH} mouvements")
    print()
    
    global best_sequence, best_score
    best_sequence = []
    best_score = -1
    
    final_state, final_score = iterative_nested(initial_state, level, max_iterations=200)
    
    print(f"\nRÉSULTATS FINAUX:")
    print(f"Séquence finale: {final_state}")
    print(f"Score final: {final_score}/{DEPTH}")
    print(f"Pourcentage de mouvements à gauche: {(final_score/DEPTH)*100:.1f}%")
    
    # Analyse de la séquence
    print(f"\nANALYSE DE LA SÉQUENCE:")
    left_moves = [i for i, move in enumerate(final_state) if move == 0]
    right_moves = [i for i, move in enumerate(final_state) if move == 1]
    print(f"Positions des mouvements à gauche: {left_moves}")
    print(f"Positions des mouvements à droite: {right_moves}")

if __name__ == "__main__":
    # Initialiser le générateur aléatoire pour la reproductibilité
    random.seed(42)
    
    print("NESTED MONTE CARLO SEARCH - PROBLÈME LEFT MOVE")
    print("=" * 50)
    
    # Choisir le mode d'exécution
    print("Modes disponibles:")
    print("1. Démonstration interactive")
    print("2. Test comparatif complet")
    print("3. Recherche unique personnalisée")
    
    try:
        #choice = input("\nChoisissez un mode (1-3): ").strip()
        choice = "2"


        if choice == "1":
            demo_single_search()
        elif choice == "2":
            test_all_methods()
        elif choice == "3":
            #level = int(input("Niveau NMCS (1-3): "))
            #iterations = int(input("Nombre d'itérations: "))
            level = 2
            iterations = 100
            global best_sequence, best_score
            best_sequence = []
            best_score = -1
            
            result, score_result = iterative_nested([], level, iterations)
            print(f"\nRésultat: {result}")
            print(f"Score: {score_result}/{DEPTH}")
        else:
            print("Choix invalide. Lancement de la démonstration par défaut.")
            demo_single_search()
            
    except (ValueError, KeyboardInterrupt):
        print("\nArrêt du programme.")
        print("Lancement de la démonstration par défaut...")
        demo_single_search()

NESTED MONTE CARLO SEARCH - PROBLÈME LEFT MOVE
Modes disponibles:
1. Démonstration interactive
2. Test comparatif complet
3. Recherche unique personnalisée
TEST DU PROBLÈME LEFT MOVE (profondeur 10)

1. PLAYOUT ALÉATOIRE:
Score moyen: 4.86
Meilleur score: 9

2. NESTED MONTE CARLO SEARCH (Niveau 1):
Démarrage de NMCS niveau 1 avec 50 itérations...
Itération 1: Nouveau meilleur score = 9
Séquence: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
Itération 2: Nouveau meilleur score = 10
Séquence: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Première séquence trouvée: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Démarrage de NMCS niveau 1 avec 50 itérations...
Itération 1: Nouveau meilleur score = 10
Séquence: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Démarrage de NMCS niveau 1 avec 50 itérations...
Itération 1: Nouveau meilleur score = 9
Séquence: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
Itération 3: Nouveau meilleur score = 10
Séquence: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Démarrage de NMCS niveau 1 avec 50 itérations...
Itération 1: Nouveau meilleur score = 